In [5]:
import pandas as pd
import requests
import os
from nba_api.stats.static import players,teams


series=pd.read_csv('playoff_series.csv')
series=series[series.year>=2014]

frames=[]
for year in range(2014,2025):
    frame=pd.read_csv(str(year)+'.csv')
    frame2=pd.read_csv(str(year)+'_2.csv')
    joined_columns = set(frame2.columns) - set(frame.columns)
    joined_columns = list(joined_columns)
    joined_columns.append('PLAYER_ID')
    joined_columns.append('date')
    frame2 = frame2[joined_columns]
    frame=frame.merge(frame2,on=['PLAYER_ID','date'],how='left')
    frames.append(frame)
df=pd.concat(frames)
df.drop_duplicates(inplace=True)
print(len(df))
def get_dates():
    dates=[]
    for year in range(2014,2025):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = '../team/'+str(year)+'ps/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['PLAYER_ID','TEAM_ID','HTM','VTM','GAME_DATE']]
                df.rename(columns={'GAME_DATE':'date'},inplace=True)
                df.drop_duplicates(inplace=True)
                dates.append(df)
    return pd.concat(dates)
dates=get_dates()
df=df.merge(dates,how='left').reset_index(drop=True)
home=df[df.HTM==df.TEAM_ABBREVIATION].reset_index(drop=True)
home.drop(columns='HTM',inplace=True)

home.rename(columns={'VTM':'opp_team'},inplace=True)


home

away=df[df.VTM==df.TEAM_ABBREVIATION].reset_index(drop=True)
away.drop(columns='VTM',inplace=True)
away.rename(columns={'HTM':'opp_team'},inplace=True)

away

none=df[df.HTM.isna()].reset_index(drop=True)
#none.drop(columns=['VTM','HTM'],inplace=True)
df=pd.concat([home,away,none])


oppframe=df[['TEAM_ID','date','opp_team']]
oppframe.dropna(subset='opp_team',inplace=True)
oppframe.drop_duplicates(inplace=True)
df.drop(columns='opp_team',inplace=True)

#df=df.merge(oppframe)
df

df=df.merge(oppframe,on=['TEAM_ID','date'],how='left')
df

19548


/tmp/ipykernel_3718/3704375478.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppframe.dropna(subset='opp_team',inplace=True)
/tmp/ipykernel_3718/3704375478.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oppframe.drop_duplicates(inplace=True)


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,MID_FG_PCT,RIGHT_CORNER_3_FGA,FGM_5_9,ITP_FGM,FGA_30_34,FGP_5_9,FGA_20_24,HTM,VTM,opp_team
0,101161,Amir Johnson,Amir,1610612761,TOR,27.0,1,0,1,0.0,...,0.000,0.0,0.0,1.0,NaN,0.0,0.0,NaN,NaN,BKN
1,201933,Blake Griffin,Blake,1610612746,LAC,25.0,1,0,1,0.0,...,0.250,0.0,0.0,1.0,NaN,0.0,1.0,NaN,NaN,GSW
2,201228,C.J. Watson,C.J.,1610612754,IND,30.0,1,0,1,0.0,...,0.000,NaN,0.0,0.0,NaN,0.0,0.0,NaN,NaN,ATL
3,2406,Caron Butler,Caron,1610612760,OKC,34.0,1,1,0,1.0,...,0.250,0.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,MEM
4,101108,Chris Paul,Chris,1610612746,LAC,29.0,1,0,1,0.0,...,0.333,0.0,0.0,1.0,NaN,0.0,2.0,NaN,NaN,GSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19543,1641716,Jarace Walker,Jarace,1610612754,IND,20.0,1,1,0,1.0,...,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,MIL
19544,203648,Thanasis Antetokounmpo,Thanasis,1610612749,MIL,31.0,1,0,1,0.0,...,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,IND
19545,1630188,Jalen Smith,Jalen,1610612754,IND,24.0,1,1,0,1.0,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,NYK
19546,1641716,Jarace Walker,Jarace,1610612754,IND,20.0,1,1,0,1.0,...,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NYK


In [6]:
none2=df[df.opp_team.isna()]
df=df.dropna(subset='opp_team')
teammap=dict(zip(df['TEAM_ABBREVIATION'],df['TEAM_ID']))

df['opp_id']=df['opp_team'].map(teammap)
df.to_csv('playoff_data.csv',index=False)

/tmp/ipykernel_3718/2211292126.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['opp_id']=df['opp_team'].map(teammap)


In [7]:
df[df.TEAM_ABBREVIATION=='DET']
df['team']=df['TEAM_ABBREVIATION']

/tmp/ipykernel_3718/2916735606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['team']=df['TEAM_ABBREVIATION']


In [8]:
df.sort_values(by='date',inplace=True)

df['series_key']=df['team']+'_'+df['opp_team']+'_'+df['year'].astype(str)


/tmp/ipykernel_3718/1847432245.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values(by='date',inplace=True)
/tmp/ipykernel_3718/1847432245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['series_key']=df['team']+'_'+df['opp_team']+'_'+df['year'].astype(str)


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,ITP_FGM,FGA_30_34,FGP_5_9,FGA_20_24,HTM,VTM,opp_team,opp_id,team,series_key
0,101161,Amir Johnson,Amir,1610612761,TOR,27.0,1,0,1,0.0,...,1.0,NaN,0.0,0.0,NaN,NaN,BKN,1610612751,TOR,TOR_BKN_2014
8918,2034,Mike Miller,Mike,1610612763,MEM,34.0,1,0,1,0.0,...,0.0,0.0,0.0,2.0,NaN,NaN,OKC,1610612760,MEM,MEM_OKC_2014
8919,203118,Mike Scott,Mike,1610612737,ATL,25.0,1,1,0,1.0,...,0.0,NaN,0.0,0.0,NaN,NaN,IND,1610612754,ATL,ATL_IND_2014
8920,203141,Mirza Teletovic,Mirza,1610612751,BKN,28.0,1,1,0,1.0,...,0.0,NaN,0.0,5.0,NaN,NaN,TOR,1610612761,BKN,BKN_TOR_2014
8921,200794,Paul Millsap,Paul,1610612737,ATL,29.0,1,1,0,1.0,...,2.0,0.0,0.5,3.0,NaN,NaN,IND,1610612754,ATL,ATL_IND_2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19472,1629052,Oshae Brissett,Oshae,1610612738,BOS,26.0,1,1,0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,DAL,1610612742,BOS,BOS_DAL_2024
19471,1641765,Olivier-Maxence Prosper,Olivier-Maxence,1610612742,DAL,21.0,1,0,1,0.0,...,0.0,NaN,0.0,0.0,NaN,NaN,BOS,1610612738,DAL,DAL_BOS_2024
19470,1628436,Luke Kornet,Luke,1610612738,BOS,28.0,1,1,0,1.0,...,NaN,NaN,NaN,0.0,NaN,NaN,DAL,1610612742,BOS,BOS_DAL_2024
17404,1630182,Josh Green,Josh,1610612742,DAL,23.0,1,0,1,0.0,...,0.0,0.0,0.0,3.0,NaN,NaN,BOS,1610612738,DAL,DAL_BOS_2024


In [11]:

series_index=df[['series_key','team','opp_team','TEAM_ID','opp_id','year']]
series_index.drop_duplicates(inplace=True)
series_index.to_csv('series_index.csv',index=False)

/tmp/ipykernel_3718/711117300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series_index.drop_duplicates(inplace=True)


In [5]:
for key in df['series_key'].unique():
    series=df[df.series_key==key]
    series.to_csv('data/'+key+'.csv',index=False)

In [6]:
series=pd.read_csv('https://raw.githubusercontent.com/gabriel1200/nba_rapm/main/playoff_series.csv')
series.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'AGE', 'GP', 'W', 'L', 'W_PCT',
       ...
       'OPP_TS_PCT', 'rs_OPP_OFF_RATING', 'rs_OPP_DEF_RATING',
       'rs_OPP_ROFF_RATING', 'rs_OPP_RDEF_RATING', 'year_avg_ts',
       'rOPP_TS_PCT', 'rTS_PCT', 'ADJ_OFF_RATING', 'ADJ_DEF_RATING'],
      dtype='object', length=137)

In [7]:
series['OPP_ID']

series=series[['PLAYER_ID','TEAM_ID','OPP_ID']]
series.to_csv('series_index.csv',index=False)